## 1. Real-World Application Structure

In [1]:
# app/config.py - Logging Configuration
import logging
import logging.handlers
import os
from datetime import datetime

class LogConfig:
    """Centralized logging configuration for the application"""
    
    # Log directory setup
    LOG_DIR = 'logs'
    os.makedirs(LOG_DIR, exist_ok=True)
    
    # Format
    DETAILED_FORMAT = (
        '[%(asctime)s] [%(name)s] [%(levelname)s] '
        '[%(funcName)s:%(lineno)d] - %(message)s'
    )
    
    SIMPLE_FORMAT = '[%(levelname)s] %(message)s'
    
    @classmethod
    def setup_logging(cls, app_name='app', level=logging.INFO):
        """Setup logging for the application"""
        root_logger = logging.getLogger()
        root_logger.setLevel(level)
        
        # Console handler
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.DEBUG)
        console_formatter = logging.Formatter(cls.SIMPLE_FORMAT)
        console_handler.setFormatter(console_formatter)
        root_logger.addHandler(console_handler)
        
        # File handler with rotation
        log_file = os.path.join(cls.LOG_DIR, f'{app_name}.log')
        file_handler = logging.handlers.RotatingFileHandler(
            log_file,
            maxBytes=10*1024*1024,  # 10MB
            backupCount=5
        )
        file_handler.setLevel(logging.DEBUG)
        file_formatter = logging.Formatter(cls.DETAILED_FORMAT)
        file_handler.setFormatter(file_formatter)
        root_logger.addHandler(file_handler)
        
        # Error file handler
        error_file = os.path.join(cls.LOG_DIR, f'{app_name}_error.log')
        error_handler = logging.FileHandler(error_file)
        error_handler.setLevel(logging.ERROR)
        error_handler.setFormatter(file_formatter)
        root_logger.addHandler(error_handler)
        
        return root_logger

# Usage
LogConfig.setup_logging('myapp')

<RootLogger root (INFO)>

## 2. Module-Level Logging

In [2]:
# app/database.py
import logging
import time
from typing import List, Dict

logger = logging.getLogger(__name__)

class DatabaseManager:
    """Database operations with comprehensive logging"""
    
    def __init__(self, connection_string: str):
        self.connection_string = connection_string
        logger.info(f"Initializing DatabaseManager with {connection_string}")
    
    def connect(self) -> bool:
        """Establish database connection"""
        try:
            logger.debug("Attempting to connect to database...")
            # Simulate connection
            time.sleep(0.5)
            logger.info("Database connection established successfully")
            return True
        except Exception as e:
            logger.error(f"Failed to connect to database: {str(e)}", exc_info=True)
            return False
    
    def query(self, sql: str) -> List[Dict]:
        """Execute a database query"""
        try:
            logger.debug(f"Executing query: {sql}")
            # Simulate query execution
            results = [{"id": 1, "name": "User1"}]
            logger.info(f"Query executed. Retrieved {len(results)} rows")
            return results
        except Exception as e:
            logger.error(f"Query failed: {str(e)}", exc_info=True)
            return []
    
    def close(self):
        """Close database connection"""
        logger.info("Closing database connection")

# Test
if __name__ == "__main__":
    db = DatabaseManager("postgresql://localhost/mydb")
    db.connect()
    db.query("SELECT * FROM users")
    db.close()

[INFO] Initializing DatabaseManager with postgresql://localhost/mydb
[INFO] Database connection established successfully
[INFO] Query executed. Retrieved 1 rows
[INFO] Closing database connection


## 3. API Service with Logging

In [3]:
# app/api.py
import logging
import time
from typing import Dict, Any

logger = logging.getLogger(__name__)

class APIService:
    """API endpoint handling with request/response logging"""
    
    def __init__(self, base_url: str):
        self.base_url = base_url
        logger.info(f"Initializing APIService with base URL: {base_url}")
    
    def handle_request(self, method: str, endpoint: str, data: Dict = None) -> Dict[str, Any]:
        """Handle API requests with detailed logging"""
        request_id = int(time.time() * 1000)
        
        logger.info(
            f"[Request:{request_id}] {method} {endpoint} "
            f"with data: {data if data else 'None'}"
        )
        
        try:
            start_time = time.time()
            logger.debug(f"[Request:{request_id}] Processing request...")
            
            # Simulate API processing
            if method == "GET":
                response = {"status": "success", "data": ["item1", "item2"]}
            elif method == "POST":
                response = {"status": "created", "id": 123}
            else:
                raise ValueError(f"Unsupported method: {method}")
            
            elapsed = time.time() - start_time
            logger.info(
                f"[Request:{request_id}] Response: {response['status']} "
                f"(Time: {elapsed:.2f}s)"
            )
            return response
            
        except Exception as e:
            logger.error(
                f"[Request:{request_id}] Error processing request: {str(e)}",
                exc_info=True
            )
            return {"status": "error", "message": str(e)}

# Test
if __name__ == "__main__":
    api = APIService("https://api.example.com")
    api.handle_request("GET", "/users")
    api.handle_request("POST", "/users", {"name": "John"})

[INFO] Initializing APIService with base URL: https://api.example.com
[INFO] [Request:1766539629041] GET /users with data: None
[INFO] [Request:1766539629041] Response: success (Time: 0.00s)
[INFO] [Request:1766539629048] POST /users with data: {'name': 'John'}
[INFO] [Request:1766539629048] Response: created (Time: 0.00s)


## 4. Performance Monitoring with Logging

In [4]:
import logging
import time
from functools import wraps
from typing import Callable, Any

logger = logging.getLogger(__name__)

def log_performance(func: Callable) -> Callable:
    """Decorator to log function execution time"""
    @wraps(func)
    def wrapper(*args, **kwargs) -> Any:
        start_time = time.time()
        logger.debug(f"Starting execution of {func.__name__}")
        
        try:
            result = func(*args, **kwargs)
            elapsed = time.time() - start_time
            
            if elapsed > 1.0:  # Log slow operations
                logger.warning(
                    f"{func.__name__} took {elapsed:.2f}s (slow operation)"
                )
            else:
                logger.debug(
                    f"{func.__name__} completed in {elapsed:.2f}s"
                )
            return result
            
        except Exception as e:
            elapsed = time.time() - start_time
            logger.error(
                f"{func.__name__} failed after {elapsed:.2f}s: {str(e)}",
                exc_info=True
            )
            raise
    
    return wrapper

# Usage examples
@log_performance
def fast_operation():
    time.sleep(0.1)
    return "Result"

@log_performance
def slow_operation():
    time.sleep(1.5)
    return "Slow Result"

@log_performance
def failing_operation():
    raise ValueError("Something went wrong")

# Test
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.DEBUG,
        format='[%(levelname)s] %(message)s'
    )
    
    fast_operation()
    slow_operation()
    # failing_operation()  # Uncomment to test error logging

[WARNING] slow_operation took 1.50s (slow operation)


## 5. Logging with Context Variables

In [5]:
import logging
import uuid
from contextvars import ContextVar

# Context variable for request ID
request_id_var: ContextVar[str] = ContextVar('request_id', default='N/A')

class ContextFilter(logging.Filter):
    """Add context information to log records"""
    
    def filter(self, record):
        record.request_id = request_id_var.get()
        return True

# Setup logging with context
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

handler = logging.StreamHandler()
formatter = logging.Formatter(
    '[%(request_id)s] %(levelname)s - %(message)s'
)
handler.setFormatter(formatter)
handler.addFilter(ContextFilter())
logger.addHandler(handler)

# Usage
def process_request():
    request_id = str(uuid.uuid4())[:8]
    request_id_var.set(request_id)
    
    logger.info("Request started")
    logger.debug("Processing data...")
    logger.info("Request completed")

if __name__ == "__main__":
    process_request()
    print()
    process_request()

[b2fda5fb] INFO - Request started
[INFO] Request started
[b2fda5fb] DEBUG - Processing data...
[DEBUG] Processing data...
[b2fda5fb] INFO - Request completed
[INFO] Request completed
[d0d39362] INFO - Request started
[INFO] Request started
[d0d39362] DEBUG - Processing data...
[DEBUG] Processing data...
[d0d39362] INFO - Request completed
[INFO] Request completed


## 6. Configuration File Based Logging

In [6]:
import logging.config
import json

# Example logging configuration (as dict)
LOGGING_CONFIG = {
    'version': 1,
    'disable_existing_loggers': False,
    'formatters': {
        'standard': {
            'format': '%(asctime)s [%(levelname)s] %(name)s: %(message)s'
        },
        'detailed': {
            'format': '[%(asctime)s] [%(levelname)s] [%(name)s] [%(funcName)s:%(lineno)d] %(message)s'
        }
    },
    'handlers': {
        'console': {
            'class': 'logging.StreamHandler',
            'level': 'DEBUG',
            'formatter': 'standard',
            'stream': 'ext://sys.stdout'
        },
        'file': {
            'class': 'logging.handlers.RotatingFileHandler',
            'level': 'DEBUG',
            'formatter': 'detailed',
            'filename': 'logs/app.log',
            'maxBytes': 10485760,
            'backupCount': 5
        }
    },
    'loggers': {
        'app': {
            'level': 'DEBUG',
            'handlers': ['console', 'file'],
            'propagate': False
        },
        'app.database': {
            'level': 'DEBUG',
            'handlers': ['console', 'file']
        },
        'app.api': {
            'level': 'INFO',
            'handlers': ['console', 'file']
        }
    },
    'root': {
        'level': 'INFO',
        'handlers': ['console', 'file']
    }
}

# Apply configuration
logging.config.dictConfig(LOGGING_CONFIG)

logger = logging.getLogger('app')
logger.info("Logging configured from dictionary")

2025-12-24 06:57:10,735 [INFO] app: Logging configured from dictionary


## 7. Best Practices Summary

1. **Use named loggers** - Always use `logging.getLogger(__name__)`
2. **Centralize configuration** - Use a config file or central setup function
3. **Include context** - Add request IDs, user IDs, etc. to logs
4. **Log at appropriate levels** - DEBUG for development, INFO for events, WARNING/ERROR for issues
5. **Include exception info** - Use `exc_info=True` when logging exceptions
6. **Rotate log files** - Use RotatingFileHandler to manage log file size
7. **Format for readability** - Include timestamps, function names, and line numbers
8. **Monitor performance** - Log execution times for slow operations
9. **Separate error logs** - Maintain separate error.log files
10. **Never log sensitive data** - Exclude passwords, tokens, and personal information